In [7]:
import pandas as pd
import requests
from tqdm import tqdm
from pathlib import Path
import os

In [8]:
os.getcwd()

'/Users/colleenking/Documents/college/programming_for_data_analytics/project'

In [10]:
data_dir = Path("./data")

if not data_dir.exists():
    os.mkdir(data_dir)
    os.mkdir(data_dir / "weather_data")

Read in the details of all weather stations - some lines aren't parsed correctly so these are skipped

We get the county, station name, the station id (station name), the height above sea level, location both in easting/northing and latitude/longitude, the open and close years

In [24]:
station_details_url = "http://cli.fusio.net/cli/climate_data/webdata/StationDetails.csv"

station_df = pd.read_csv(station_details_url, on_bad_lines='skip')

station_df.head()

,county,station name,name,height(m),easting,northing,latitude,longitude,open year,close year
0,Antrim,5880,LH_RATHLIN_WEST,10,309200,451800,55.30083,-6.28028,2000,(null)
1,Carlow,4415,TULLOW (Waterworks),76,284700,173400,52.80528,-6.74306,1985,(null)
2,Carlow,2414,BORRIS G.S.,85,272400,150700,52.60278,-6.93056,1944,1991
3,Carlow,1214,CARLOW (SUGAR FACTORY),58,272200,178400,52.85139,-6.92778,1953,1960
4,Carlow,115,HACKETSTOWN RECTORY,182,297600,180500,52.86667,-6.55000,1910,1944


In [14]:
station_df.dtypes

county           object
station name      int64
name             object
height(m)         int64
easting           int64
northing          int64
latitude        float64
longitude       float64
open year        object
close year       object
dtype: object

The open and close years are stored as stings, but we want them as ints

In [26]:
station_df["close year"] = station_df["close year"].replace('(null)', None).astype('Int64')
station_df["open year"] = station_df["open year"].replace('(null)', None).astype('Int64')
station_df.head()

,county,station name,name,height(m),easting,northing,latitude,longitude,open year,close year
0,Antrim,5880,LH_RATHLIN_WEST,10,309200,451800,55.30083,-6.28028,2000,<NA>
1,Carlow,4415,TULLOW (Waterworks),76,284700,173400,52.80528,-6.74306,1985,<NA>
2,Carlow,2414,BORRIS G.S.,85,272400,150700,52.60278,-6.93056,1944,1991
3,Carlow,1214,CARLOW (SUGAR FACTORY),58,272200,178400,52.85139,-6.92778,1953,1960
4,Carlow,115,HACKETSTOWN RECTORY,182,297600,180500,52.86667,-6.55000,1910,1944


In [27]:
station_df.dtypes

county           object
station name      int64
name             object
height(m)         int64
easting           int64
northing          int64
latitude        float64
longitude       float64
open year         Int64
close year        Int64
dtype: object

In [28]:
station_df.to_csv(data_dir / "stations.csv", index=False)

We can use the station id to download daily data automatically because the download url is predictable - replacing the station id gives the filename we're looking for (https://cli.fusio.net/cli/climate_data/webdata/dly[station_id].csv)

We also don't care about closed stations, so we only need the rows where close year is null

In [34]:
open_stations = list(station_df[~pd.isna(station_df["close year"])]["station name"])
len(open_stations)

1550

1550 is a lot of stations - each file is about 2mb, so in total this would be around 3Gb if we naively downloaded it all. We should just get a sample a few from each county instead.

5 stations * 26 counties * 2mb gives around 260mb, which is much more reasonable

In [39]:
# https://stackoverflow.com/questions/22472213/python-random-selection-per-group

sample_stations = station_df[~pd.isna(station_df["close year"])].groupby('county').apply(lambda x: x.sample(5)).reset_index(drop=True)

print(len(sample_stations))
sample_stations.head()


130


/var/folders/s7/bsy4w2n54pvcvlbx9jg9h2040000gn/T/ipykernel_13248/2311088952.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_stations = station_df[~pd.isna(station_df["close year"])].groupby('county').apply(lambda x: x.sample(5)).reset_index(drop=True)


,county,station name,name,height(m),easting,northing,latitude,longitude,open year,close year
0,Carlow,6014,CLASHGANNA MILLS,27,273800,146200,52.56250,-6.91111,1987,1989
1,Carlow,4814,CARLOW (Oak Park),61,273000,179500,52.86111,-6.91528,1967,1996
2,Carlow,515,TULLOW (MT.ST.JOSEPH'S),79,285800,171800,52.79028,-6.72778,1949,1960
3,Carlow,5314,BAGENALSTOWN (FENAGH II),104,277700,161500,52.69917,-6.85111,1982,1989
4,Carlow,6914,GARRYHILL (MILLTOWN),107,278600,158700,52.67389,-6.83750,2003,2005


In [41]:
# url format: https://cli.fusio.net/cli/climate_data/webdata/dly875.csv
for station in tqdm(list(sample_stations['station name'])):
    try:
        url = f"https://cli.fusio.net/cli/climate_data/webdata/dly{station}.csv"
        response = requests.get(url)
        assert response.status_code == 200
        with open(data_dir / f"weather_data/{station}.csv", "wb") as f:
            f.write(response.content)
    except:
        print(f"Couldn't retrieve data for station {station}")

  2%|▋                                          | 2/130 [00:00<00:23,  5.44it/s]

Couldn't retrieve data for station 6014
Couldn't retrieve data for station 4814


  3%|█▎                                         | 4/130 [00:00<00:21,  5.92it/s]

Couldn't retrieve data for station 515
Couldn't retrieve data for station 5314


  5%|█▉                                         | 6/130 [00:01<00:21,  5.80it/s]

...

Couldn't retrieve data for station 508
Couldn't retrieve data for station 1520


 98%|████████████████████████████████████████▎| 128/130 [00:24<00:00,  6.19it/s]

Couldn't retrieve data for station 2515
Couldn't retrieve data for station 1824


100%|█████████████████████████████████████████| 130/130 [00:24<00:00,  5.32it/s]

Couldn't retrieve data for station 3424
Couldn't retrieve data for station 815


It looks like a lot of the stations' data isn't available via these urls - of the 130 expected, only 7 were actually downloaded, or a little over 5%. If that holds true across the whole dataset, 5% of the 1550 stations should give us an expected 83 files, totalling ~160mb, which would be manageable.

In [44]:
1550*(7/130)

83.46153846153847

In [42]:
# this station gives a 404 Not Found status code - the file doesn't exist at this url
pd.read_csv(f"https://cli.fusio.net/cli/climate_data/webdata/dly6014.csv")

HTTPError: HTTP Error 404: Not Found

In [45]:
files_downloaded = 0
for station in tqdm(open_stations):
    try:
        url = f"https://cli.fusio.net/cli/climate_data/webdata/dly{station}.csv"
        response = requests.get(url)
        assert response.status_code == 200
        with open(data_dir / f"weather_data/{station}.csv", "wb") as f:
            f.write(response.content)
        files_downloaded += 1
    except:
        pass
print(f'Downloaded {files_downloaded} files')

100%|███████████████████████████████████████| 1550/1550 [03:15<00:00,  7.94it/s]

Downloaded 38 files


We only got 38 files in the end, we may have gotten lucky with the sample that had a hit rate of 5%.

Now that we have all our files downloaded, we need to read them into pandas so we can analyse them.
Each file has a header that we need to skip, however the number of rows changes from file to file. The row we want always starts with "date", so we can use this 
We also need to keep track of the station id so 